In [1]:
import pandas as pd
import altair as alt
from astropy.time import Time

In [103]:
df = pd.read_csv("data/Near-Earth_Comets_Orbital_Elements_rows.csv")
df['Epoch (TDB)'] = pd.to_numeric(df['Epoch (TDB)'], errors='coerce')

# Assume Epoch is MJD or short-form JD, convert to full JD
df['Julian_Date'] = df['Epoch (TDB)'] + 2400000.5

# Now convert to datetime from Julian
JULIAN_UNIX_OFFSET = 2440587.5  # JD at Unix epoch
df['Date'] = pd.to_timedelta(df['Julian_Date'] - JULIAN_UNIX_OFFSET, unit='D') + pd.Timestamp("1970-01-01")
df

,Object,Epoch (TDB),TP (TDB),e,i (deg),w (deg),Node (deg),q (AU),Q (AU),P (yr),MOID (AU),A1 (AU/d^2),A2 (AU/d^2),A3 (AU/d^2),DT (d),ref,Object_name,Julian_Date,Date
0,P/2004 R1 (McNaught),54629,2455248.548,0.682527,4.894556,0.626838,295.985450,0.986192,5.23,5.48,0.027011,NaN,NaN,NaN,NaN,20,P/2004 R1 (McNaught),2454629.5,2008-06-12
1,P/2008 S1 (Catalina-McNaught),55101,2454741.329,0.666313,15.100746,203.649023,111.392003,1.190642,5.95,6.74,0.194101,NaN,NaN,NaN,NaN,13,P/2008 S1 (Catalina-McNaught),2455101.5,2009-09-27
2,1P/Halley,49400,2446467.395,0.967143,162.262691,111.332485,58.420081,0.585978,35.08,75.32,0.063782,2.700000e-10,1.550000e-10,NaN,NaN,J863/77,1P/Halley,2449400.5,1994-02-17
3,2P/Encke,56870,2456618.204,0.848268,11.779995,186.540346,334.569806,0.336092,4.09,3.30,0.173092,1.580000e-10,-5.050000e-12,NaN,NaN,74,2P/Encke,2456870.5,2014-08-01
4,3D/Biela,-9480,2390514.115,0.751299,13.216400,221.658800,250.669000,0.879073,6.19,6.65,0.000518,3.900000e-09,-2.540000e-10,NaN,NaN,IAUCAT03,3D/Biela,2390520.5,1832-12-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,P/2014 U2 (Kowalski),56974,2456937.650,0.602949,7.495593,35.252210,356.180679,1.164298,4.70,5.02,0.183671,NaN,NaN,NaN,NaN,10,P/2014 U2 (Kowalski),2456974.5,2014-11-13
156,P/2015 A3 (PANSTARRS),57040,2457076.250,0.848168,172.513745,249.508614,277.082142,1.153857,14.05,20.95,0.204591,NaN,NaN,NaN,NaN,1,P/2015 A3 (PANSTARRS),2457040.5,2015-01-18
157,C/2015 D1 (SOHO),57073,2457073.248,0.994268,69.616289,234.953017,95.881385,0.028321,9.85,10.98,0.515916,1.250000e-06,NaN,NaN,NaN,3,C/2015 D1 (SOHO),2457073.5,2015-02-20
158,C/2015 F2 (Polonia),57110,2457141.343,0.962399,28.661030,352.058758,262.481614,1.209544,63.13,182.45,0.198704,NaN,NaN,NaN,NaN,6,C/2015 F2 (Polonia),2457110.5,2015-03-29


In [104]:
df = df[df['Date'] > '1990']
df = df[df['P (yr)'] < 100]
df['Decade'] = (df['Date'].dt.year // 10) * 10

In [105]:
alt.Chart(df).mark_circle().encode(
    x = alt.X("Date:T"),
    y = alt.Y("P (yr):Q")
)

alt.Chart(...)

In [106]:
alt.Chart(df).mark_circle().encode(
    x = alt.X("Date:T"),
    y = alt.Y("P (yr):Q"),
    size = alt.Size("MOID (AU):Q")
)

alt.Chart(...)

In [97]:
alt.Chart(df).mark_circle().encode(
    x = alt.X("q (AU):Q"),
    y = alt.Y("P (yr):Q"),
    size = alt.Size("MOID (AU):Q"),
    color = alt.Color("e:Q"),
    row = alt.Row("Decade:O")
)

alt.Chart(...)

In [98]:
selection = alt.selection_point(on="mouseover")

alt.Chart(df).mark_circle().add_params(
    selection
).encode(
    x = alt.X("q (AU):Q"),
    y = alt.Y("P (yr):Q"),
    size = alt.Size("MOID (AU):Q"),
    column = alt.Column("Decade:O"),
    color = alt.condition(selection, "e:Q", alt.value("lightgray")),
    tooltip = ("q (AU):Q", "P (yr):Q")
).properties(width = 250)

alt.Chart(...)

In [112]:
selection = alt.selection_point(on="mouseover")
bind = alt.selection_interval(bind='scales', encodings=["x"])

alt.Chart(df).mark_circle().add_params(
    bind,
    selection
).encode(
    x = alt.X("q (AU):Q", title = "Perehelion Distance (AU)"),
    y = alt.Y("P (yr):Q", title = "Orbital Period (yrs)"),
    size = alt.Size("MOID (AU):Q", title = "Distance from Earth (AU)"),
    #column = alt.Column("Decade:O"),
    color = alt.condition(selection, "e:Q", alt.value("lightgray"), scale = alt.Scale(range="diverging"), title="Eccentricity"),
    tooltip = ("Date", "q (AU):Q", "P (yr):Q"),
    #opacity = alt.value(0.5)
).properties(
    width = 500,
    title = "What physical properties of near-Earth Comets influence their orbital period?"
)

alt.Chart(...)

In [134]:
selection = alt.selection_point(on="mouseover")
bind = alt.selection_interval(bind='scales', encodings=["x"])

alt.Chart(df).mark_circle().add_params(
    bind,
    selection
).encode(
    x = alt.X("q (AU):Q", title = "Perehelion Distance (AU)", axis = alt.Axis(grid=False)),
    y = alt.Y("P (yr):Q", title = "Orbital Period (yrs)"),
    size = alt.Size("MOID (AU):Q", scale=alt.Scale(reverse=True), title = "Distance from Earth (AU)"),
    #column = alt.Column("Decade:O"),
    color = alt.condition(selection, "e:Q", alt.value("lightgray"), scale = alt.Scale(scheme = "redblue"), title="Eccentricity"),
    tooltip = ("Date", "q (AU):Q", "P (yr):Q"),
    opacity = alt.value(1.25)
).properties(
    width = 600,
    title = "What physical properties of near-Earth Comets influence their orbital period?"
)

alt.Chart(...)